### Import Libraries 

In [1]:
import requests
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np

### Web page scraped

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
print(page.status_code)

200


In [4]:
#Check the length of the first 20 rows
[len(T) for T in tr_elements[:20]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name1=t.text_content()
    #print (name1)
    name=name1.replace('\n','')
    #print (name)
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))
    

1:"Postcode"
2:"Borough"
3:"Neighbourhood"


In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data1=t.text_content()
        data=data1.replace('\n','')
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]

[287, 287, 287]

### Data transformed into pandas dataframe

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [9]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


### Ignore cells with a borough that is Not assigned

In [10]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

### More than one neighborhood can exist in one postal code area, combined into one row with the neighborhoods separated with a comma

In [11]:
df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))

In [12]:
df1.head(10)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
M1E,Scarborough,"West Hill,Guildwood,Morningside"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview"
M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"


In [13]:
df2 = df1.reset_index()

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [14]:
df1.loc[df1["Neighbourhood"]=="Not assigned","Neighbourhood"]=df1.loc[df1["Neighbourhood"]=="Not assigned","Borough"]

In [15]:
df1.shape

(103, 2)

In [16]:
df2['Borough']= df2['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df2.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [17]:
df2.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
2,M1E,Scarborough,"West Hill,Guildwood,Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview"
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


### Used the csv file to create the requested dataframe

In [18]:
data = pd.read_csv('https://cocl.us/Geospatial_data')

In [19]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Rename the columns 

In [25]:
data.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [26]:
data.head(10)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Merge two dataframes to create the requested dataframe

In [27]:
data1 = pd.merge(df2, data, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=True, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

In [29]:
data1.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
